In [91]:
import pandas as pd
import os
import warnings

pd.set_option('display.expand_frame_repr', False)
warnings.filterwarnings("ignore", message="Workbook contains no default style, apply openpyxl's default")

shop = 'Bayer'
sc_start_date = 'March 16'
sc_end_date = 'April 6'

# Order Data cannot be extracted from Lazada Seller Center in one go, so we had to break it down into multiple excel files
# All excel files from the same shop are stored in one folder
folder = f'Lazada Seller Center/{shop}'
excel_files = os.listdir(folder)

# Loop over all the excel files in the folder and import only the "orderNumber" and "status" columns
dfs = {}
for file in excel_files:
    file_path = os.path.join(folder, file)
    df = pd.read_excel(file_path, usecols=["orderNumber", "status", "createTime"])
    dfs[file] = df

# The result will be one DataFrame with "orderNumber" and "status" columns for each excel file
for file, df in dfs.items():
    print(file)
    print("Number of rows:", len(df))
    print(df.head(),'\n')

Lazada Bayer (March 16 - April 6).xlsx
Number of rows: 13511
          createTime      orderNumber     status
0  06 Apr 2024 23:58  815626858923070  confirmed
1  06 Apr 2024 23:36  821582920216964  confirmed
2  06 Apr 2024 23:36  821582920216964  confirmed
3  06 Apr 2024 23:01  815594271466239  confirmed
4  06 Apr 2024 22:06  815569891264333  confirmed 



In [111]:
# Merge into one DataFrame
seller_center = pd.concat(dfs.values(), keys=dfs.keys(), ignore_index=True)
print(f"Seller Center Lazada {shop} ({sc_start_date} - {sc_end_date})")
print("Number of rows:", len(seller_center))

Seller Center Lazada Bayer (March 16 - April 6)
Number of rows: 13511


In [112]:
# Remove duplicate entries
seller_center = seller_center.drop_duplicates(subset=['orderNumber'])
print(f"Seller Center Lazada {shop} ({sc_start_date} - {sc_end_date})")
print("Number of unique rows:", len(seller_center))

Seller Center Lazada Bayer (March 16 - April 6)
Number of unique rows: 4890


In [102]:
# Import Snowflake data into a DataFrame
snowflake = pd.read_csv(f'Snowflake/Snowflake Lazada {shop}.csv', usecols=['PLFM_ORD_ID','ORD_STAT_END', 'ORD_STAT_PLFM', 'ORDER_DATE'])

# Determine date range of the dataset
date_range = pd.date_range(start=snowflake.loc[:, 'ORDER_DATE'].min(), end=snowflake.loc[:, 'ORDER_DATE'].max())
sf_start_date = date_range[0].date()
sf_end_date = date_range[-1].date()

print(f"Snowflake Lazada {shop} ({sf_start_date} - {sf_end_date})")
print("Number of rows:", len(snowflake))

# Remove duplicates
snowflake.drop_duplicates(subset=['PLFM_ORD_ID'], inplace=True, ignore_index=True)
print("Number of unique rows:", len(snowflake), '\n')
print(snowflake.head())

Snowflake Lazada Bayer (2024-03-17 - 2024-04-06)
Number of rows: 13140
Number of unique rows: 4751 

       PLFM_ORD_ID ORD_STAT_END ORD_STAT_PLFM  ORDER_DATE
0  819880775302825      shipped       shipped  2024-04-04
1  819757326405409    delivered     delivered  2024-04-04
2  810197641635216    delivered     confirmed  2024-03-27
3  805639459275365    delivered     delivered  2024-03-19
4  814153035973422    delivered     delivered  2024-04-04


In [95]:
# Merge the Seller Center and Snowflake DataFrames so we could compare them
merged_df = pd.merge(seller_center, snowflake, left_on="orderNumber", right_on="PLFM_ORD_ID", how="inner")
print(f"Lazada {shop} Seller Center vs Snowflake ({sf_start_date} - {sf_end_date})")
print(merged_df.head(), '\n')
total = len(merged_df)
print("Number of rows:", total)

# Count the number of mismatched status
mismatched_status = merged_df[merged_df["status"] != merged_df["ORD_STAT_PLFM"]]
count_mismatched_status = len(mismatched_status)
print("Number of mismatched status:", count_mismatched_status)

# Calculate percent mismatched status
percent_mismatched_status = (count_mismatched_status / total) * 100
print("Percent mismatched status:", round(percent_mismatched_status, 2), "%")


Lazada Bayer Seller Center vs Snowflake (2024-03-17 - 2024-04-06)
          createTime      orderNumber     status      PLFM_ORD_ID ORD_STAT_END ORD_STAT_PLFM  ORDER_DATE
0  06 Apr 2024 23:58  815626858923070  confirmed  815626858923070    delivered     delivered  2024-04-06
1  06 Apr 2024 23:36  821582920216964  confirmed  821582920216964    delivered     delivered  2024-04-06
2  06 Apr 2024 23:01  815594271466239  confirmed  815594271466239    delivered     delivered  2024-04-06
3  06 Apr 2024 22:06  815569891264333  confirmed  815569891264333    delivered     delivered  2024-04-06
4  06 Apr 2024 22:02  815593034795114  confirmed  815593034795114    delivered     delivered  2024-04-06 

Number of rows: 4751
Number of mismatched status: 3012
Percent mismatched status: 63.4 %


In [104]:
# Check possible values of status column and count 
print(f"Seller Center Lazada {shop}")
print("Number of unique rows:", len(seller_center), '\n')
seller_center_status = seller_center['status'].value_counts()
print(seller_center_status, '\n')
print()
print(f"Snowflake Lazada {shop}")
print("Number of unique rows:", len(snowflake), '\n')
snowflake_ORD_STAT_PLFM = snowflake['ORD_STAT_PLFM'].value_counts()
print(snowflake_ORD_STAT_PLFM, '\n')
print()
print(f"Snowflake Lazada {shop}")
print("Number of unique rows:", len(snowflake), '\n')
snowflake_ORD_STAT_END = snowflake['ORD_STAT_END'].value_counts()
print(snowflake_ORD_STAT_END)

Seller Center Lazada Bayer
Number of unique rows: 4890 

status
confirmed                          2949
delivered                          1464
canceled                            413
Package Returned                     41
In Transit: Returning to seller      11
shipped                               6
returned                              2
ready_to_ship                         2
Lost by 3PL                           1
Buyer Delivery Failed                 1
Name: count, dtype: int64 


Snowflake Lazada Bayer
Number of unique rows: 4751 

ORD_STAT_PLFM
delivered               2959
shipped                 1007
canceled                 368
confirmed                307
ready_to_ship             41
shipped_back_success      34
packed                    15
shipped_back              15
unpaid                     2
returned                   2
lost_by_3pl                1
Name: count, dtype: int64 


Snowflake Lazada Bayer
Number of unique rows: 4751 

ORD_STAT_END
delivered    3266
shipped 

In [105]:
# Get non-end status rows (unpaid, packed, shipped, shipped back, ready tp ship)
non_end_status = ['unpaid', 'packed', 'shipped','shipped_back', 'ready_to_ship']

seller_center = seller_center[seller_center['status'].isin(non_end_status)]
snowflake = snowflake[snowflake['ORD_STAT_PLFM'].isin(non_end_status)]

print(f"Seller Center Lazada {shop} Non-End Status")
print("Number of rows:", len(seller_center), '\n')
seller_center_status = seller_center['status'].value_counts()
print(seller_center_status, '\n')
print()
print(f"Snowflake Lazada {shop} Non-End Status")
print("Number of rows:", len(snowflake), '\n')
snowflake_status = snowflake['ORD_STAT_PLFM'].value_counts()
print(snowflake_status)

Seller Center Lazada Bayer Non-End Status
Number of rows: 8 

status
shipped          6
ready_to_ship    2
Name: count, dtype: int64 


Snowflake Lazada Bayer Non-End Status
Number of rows: 1080 

ORD_STAT_PLFM
shipped          1007
ready_to_ship      41
packed             15
shipped_back       15
unpaid              2
Name: count, dtype: int64


In [106]:
# Merge the Seller Center and Snowflake DataFrames so we could compare them
final_df = pd.merge(seller_center, snowflake, left_on="orderNumber", right_on="PLFM_ORD_ID", how="inner")
print(f"Lazada {shop} Seller Center vs Snowflake ({sf_start_date} - {sf_end_date})")
print(final_df.head(), '\n')
print("Number of rows:", total)

# Count the number of mismatched status
mismatched_status = final_df[final_df["status"] != final_df["ORD_STAT_PLFM"]]
count_mismatched_status = len(mismatched_status)
print("Number of mismatched status:", count_mismatched_status)

# Calculate percent mismatched status
percent_mismatched_status = (count_mismatched_status / total) * 100
print("Percent mismatched status:", round(percent_mismatched_status, 2), "%")


Lazada Bayer Seller Center vs Snowflake (2024-03-17 - 2024-04-06)
          createTime      orderNumber         status      PLFM_ORD_ID ORD_STAT_END  ORD_STAT_PLFM  ORDER_DATE
0  06 Apr 2024 11:20  821237174517469        shipped  821237174517469      shipped        shipped  2024-04-06
1  30 Mar 2024 15:59  817698101200351        shipped  817698101200351      shipped        shipped  2024-03-30
2  26 Mar 2024 14:43  815541598055059  ready_to_ship  815541598055059      shipped        shipped  2024-03-26
3  26 Mar 2024 00:13  809438803786980  ready_to_ship  809438803786980          rts  ready_to_ship  2024-03-26 

Number of rows: 4751
Number of mismatched status: 1
Percent mismatched status: 0.02 %
